In [1]:
import requests
from lxml import etree

In [2]:
response = requests.get("https://cofacts.g0v.tw/reply/A89kv2wBqwaEkHKwLTTZ")
html = etree.HTML(response.content.decode())

In [3]:
print(html.xpath("//h1/text()"))

['真的假的', '半夜闻到煤气味，开灯瞬间导致爆炸_李玉玲']


*** 

In [4]:
import requests
from lxml import etree

In [5]:
response = requests.get("https://cofacts.g0v.tw/replies?after=&before=&filter=OPINIONATED")
html = etree.HTML(response.content)
ls = html.xpath('//a[@class="jsx-3552945388 jsx-3720007368 item"]')
next_link = html.xpath('//a[@class="jsx-684928770"]/@href')[1]
total_number = html.xpath('//p[@class="jsx-3107014753"]/text()')[0]
all_ls = []
while(len(all_ls)<int(total_number)):
    for l in ls:
        all_ls.append(l.xpath(".//div[@class='jsx-3552945388 jsx-3720007368 item-text']/text()")[1])
        if(len(all_ls)%50 == 0 ):
            print(len(all_ls))
    response = requests.get("https://cofacts.g0v.tw/replies?"+next_link)
    html = etree.HTML(response.content)
    ls = html.xpath('//a[@class="jsx-3552945388 jsx-3720007368 item"]')
    next_link = html.xpath('//a[@class="jsx-684928770"]/@href')[1]

50
100
150
200
250
300
350
400
450
500
550
600
650
700
750
800
850
900
950
1000
1050
1100
1150
1200
1250
1300
1350
1400
1450
1500
1550
1600
1650
1700
1750
1800
1850
1900
1950
2000
2050
2100
2150
['德國聯邦議院電子連署平臺自 2005 年啟用以來，已進行多次修正及更多功能開發，請願頁面包含請願內容、討論區論壇、案號、請願書 PDF 檔下載、請願起始日、請願狀態、投票結果及回應\n理由、連署數量（含線上及紙本）及時間進程（以折線圖呈現），同時也導入社群分享（Twitter、Google+及 Facebook）不同於亞洲國家的請願或連署機制，多以附議或連署人數做為權責機關是否出面回應的關鍵門檻，優先處理當下民意所關注的議題，歐美國家多以請願或提議內容為導向，沒有人數限制的情況下，每一位公民所提出的訴求都能被平等看待，權責機關亦可依據議題的重要性、急迫性及可行性來選擇與回覆。德國採綜合式機制，保留上述兩者之優點，即便審查的過程需耗費較多行政成本，卻能夠提供更多機關回應的機會，從多方面了解民眾對於公共議題的想法與建議。', '不同民調機構，甚至施作時間點及地方特性，首長滿意度起起伏伏很正常，只要認真面對市民，好好做事，對地方政府首長們，不用因民調成績就給予過多苛責。', '美國國務院東亞和太平洋事務局副助理國務卿費德瑋（Jonathan Fritz）說，美國非常擔憂外國對手企圖干預選舉，且北京肯定有偏愛的候選人，美台將會合作防止外界干預台灣選舉。', '政府收購的文旦格外品有 4000 公噸，去掉可以做果醬果醋果茶蜜餞或萃取精油做清潔劑的之後之，剩下的會拿去做堆肥。\n\n國民黨團所揭露的正是堆肥場，上頭有 1000 公噸文旦，僅佔全台文旦總產量 1%，差不多就是格外品處置的比例。\n', '3D技術即是利用雙眼視差的原理，透過各種光學技術，讓左眼及右眼分別接收到不同的視角，使觀看者對畫面產生立體視覺。早期是以佩戴眼鏡的方式，例如紅藍雙色的濾色眼鏡、偏光式眼鏡。透過眼鏡的濾光，讓雙眼分別看到不同的畫面，產生立體視覺的效果。  里約大冒險的影片並沒有使用到3D裸視的做法。', '明白

In [1]:
import requests
from lxml import etree
import threading
import time
import csv
import re

In [2]:
def check_value(title):
    pipe = ["維基","Server Error","Access denied","Yahoo is now","Page Not Found","痞客邦","百度","404","Security"]
    if any(x in title for x in pipe):
        return True
    elif (re.search("[^a-zA-Z0-9\b]+",title)):
        return False
    else:
        return True

In [3]:
def clean_news(l,all_ls):
    newslink = hostName+ l
    response = requests.get(newslink);
    html = etree.HTML(response.content)
    origin_news = "".join(html.xpath("//div[@class='jsx-1919529045 message']/text()"))
    if(len(origin_news)>70 or len(origin_news)<3 or check_value(origin_news)) :
        return 0;
    new_titles = html.xpath("//section[@class='jsx-588669885 links']//article[@class='jsx-1682249194 link']/h1/@title")
    reason = "".join(html.xpath("//section[@class='jsx-3677418999 jsx-423907629 section']//div/text()"))
    for i in new_titles:
        if(check_value(i)):
            return 0
        dictionary = {}
        dictionary["link"] = newslink
        dictionary["news1"] = origin_news
        dictionary["news2"] = i
        dictionary["fact"] = False
        dictionary["reason"] = reason
        all_ls.append(dictionary)

In [4]:
hostName = "https://cofacts.g0v.tw"
response = requests.get("https://cofacts.g0v.tw/replies?after=&before=&filter=RUMOR")
html = etree.HTML(response.content)
newslink_ls = html.xpath('//a[@class="jsx-3552945388 jsx-3720007368 item"]/@href')
next_link = html.xpath('//a[@class="jsx-684928770"]/@href')[1]
total_number = html.xpath('//p[@class="jsx-3107014753"]/text()')[0]
all_ls = []
count = 0
while(True):
    threads = []
    for l in newslink_ls:
        count += 1
        threads.append(threading.Thread(target=clean_news,args=(l,all_ls)))
        threads[-1].start()
        if(count%100 == 0  and count != 0):
            print(count)
    
    for t in threads:
        t.join()

    response = requests.get("https://cofacts.g0v.tw/replies?"+next_link)
    html = etree.HTML(response.content)
    try:
        newslink_ls = html.xpath('//a[@class="jsx-3552945388 jsx-3720007368 item"]/@href')
        next_link = html.xpath('//a[@class="jsx-684928770"]/@href')[1]
    except:
        break
print("done")

100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
done


In [5]:
keys = all_ls[0].keys()
with open('fake.csv', 'w',encoding="utf8") as output_file:
    dict_writer = csv.DictWriter(output_file, keys)
    dict_writer.writeheader()
    dict_writer.writerows(all_ls)

***

In [3]:
def clean_all_data(l,all_ls):
    newslink = hostName+ l
    response = requests.get(newslink);
    html = etree.HTML(response.content)
    origin_news = "".join(html.xpath("//div[@class='jsx-1919529045 message']/text()"))
    if(len(origin_news)<3 or check_value(origin_news)) :
        return 0;
    new_titles = html.xpath("//section[@class='jsx-588669885 links']//article[@class='jsx-1682249194 link']/h1/@title")
    reason = "".join(html.xpath("//section[@class='jsx-3677418999 jsx-423907629 section']//div/text()"))
    for i in new_titles:
        if(check_value(i)):
            return 0
        all_ls.append(i)
    all_ls.append(origin_news)
    all_ls.append(reason)
#         dictionary = {}
#         dictionary["link"] = newslink
#         dictionary["news1"] = origin_news
#         dictionary["news2"] = i
#         dictionary["fact"] = False
#         dictionary["reason"] = reason
#         all_ls.append(dictionary)

In [5]:
hostName = "https://cofacts.g0v.tw"
response = requests.get("https://cofacts.g0v.tw/replies")
html = etree.HTML(response.content)
newslink_ls = html.xpath('//a[@class="jsx-3552945388 jsx-3720007368 item"]/@href')
next_link = html.xpath('//a[@class="jsx-684928770"]/@href')[1]
total_number = html.xpath('//p[@class="jsx-3107014753"]/text()')[0]
all_ls = []
count = 0
while(True):
    threads = []
    for l in newslink_ls:
        count += 1
        threads.append(threading.Thread(target=clean_all_data,args=(l,all_ls)))
        threads[-1].start()
        if(count%100 == 0  and count != 0):
            print(count)
    
    for t in threads:
        t.join()
    
    if(count>=int(total_number)):
        break

    response = requests.get("https://cofacts.g0v.tw/replies?"+next_link)
    html = etree.HTML(response.content)
    try:
        newslink_ls = html.xpath('//a[@class="jsx-3552945388 jsx-3720007368 item"]/@href')
        next_link = html.xpath('//a[@class="jsx-684928770"]/@href')[1]
    except:
        break
print("done")

100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
12100
12200
12300
12400
12500
12600
12700
12800
12900
13000
13100
13200
13300
13400
13500
13600
13700
13800
13900
14000
14100
14200
14300
14400
14500
14600
14700
14800
14900
15000
15100
15200
15300
15400
15500
15600
15700
15800
15900
16000
16100
16200
16300
16400
16500
16600
16700
16800
16900
17000
17100
17200
17300
17400
17500
17600
17700
17800
17900
18000
18100
18200
18300
18400
1850

In [15]:
with open("all_corpus.txt","w",encoding="utf8") as fw:
    for i in all_ls:
        fw.writelines(i+"\n")

In [19]:
all_[:30]

['太陽能發電比燃煤發電便宜 大陸宣布做到了 | 聯合新聞網\n',
 '人家在做、我們在譙！太陽能發電比燃煤發電便宜 大陸宣布做到了！\n',
 '新華社於2018/12/29發布新聞稿，中國新聞無從查證起。\n',
 '太陽能發電比燃煤發電便宜 大陸宣布做到了 | 聯合新聞網\n',
 '人家在做、我們在譙！太陽能發電比燃煤發電便宜 大陸宣布做到了！\n',
 '技術逐年進展，也讓太陽光電和陸域風電裝設成本下降、容量因數（Capacity Factor，可理解為全年發電比例）上升，使全生命週期的均化發電成本大跌。\n',
 '楊安澤，雲林人之子，美國台裔商人，也是第一個民主黨爭取2020年美國總統大選的提名人，也是第一位宣布競選的台裔美國人。楊安澤的父親《楊界雄》是台灣面板教父，雲林元長田洋村人，畢業於元長仁愛國小，是《虎中傑出校友》，兩岸及美國科技界尖端人才，都有《楊界雄教授》的學生。（感謝~陳榮俊特派的分享）台大物理系畢業與服役後即申請到美國印地安那州聖母大學物理系的獎學金，完成碩士學位後又以優異成績申請到柏克萊加州大學的獎學金，在該校的物理系完成了博士學位。楊界雄博士，是柏克萊加大的物理博士，獲得美國69項專利。曾任職於貝爾（Bell）實驗室，奇異(GE)研發中心和IBM華生（Watson）研究所，如此超優異的學經歷回國後便服務於瀚宇彩晶公司擔任研究中心副總，並於其領導下成功突破多年來液晶封裝專利的限制，爲全世界的光電顯示器業界投下一顆震撼彈。並於2001年榮獲國際資訊顯示學會院士(SID Fellow)。楊界雄教授，經常鼓勵年輕人說：「年輕人就是要有guts！」其優秀的公子~楊安澤，在人生旅途中，也就是《非常有guts的表現》。我相信~台裔 楊安澤，將會全力以赴來落實所參選之路，宣布参選要相當勇氣、也要相當實力，認真的男人永遠最帥！不管，結果是如何，雲林人第二代，台裔 楊安澤，肯定是最棒，最傑出的！楊安澤，是雲林之光！更是台灣之耀！好友，若是您有美國友人，請您轉達請支持~台裔美國人 楊安澤 2020年美國民主黨提名參選人。\n',
 '雲林元長鄉沒有田洋村好嗎?他是潭西村人\n',
 '廣東省原副省長萬慶良被宣判判處無期徒刑！查封519億的人民幣及財產全部歸回國庫。判決書由萬慶良自己讀！\n',
 '中國南寧市中級人民法院於2016/9/30宣判，但中國

***

In [1]:
import jieba
import pandas as pd
import csv
from langconv import *
import re

In [2]:
jieba.set_dictionary("dict.txt.big.txt")

In [3]:
all_ = []
with open("all_corpus.txt",encoding="utf8") as file:
    count = 0;
    for li in file:
        all_.append(li)

In [4]:
arr = []
with open("fake.csv",encoding="utf8") as file:
    rows = csv.reader(file)
    next(rows)
    for row in rows:
        if(len(row)>0):
            arr.append(HanziConv.toTraditional(row[1]))
            arr.append(HanziConv.toTraditional(row[2]))

with open("unfake.csv",encoding="utf8") as file:
    rows = csv.reader(file)
    next(rows)
    for row in rows:
        if(len(row)>0):
            arr.append(HanziConv.toTraditional(row[1]))
            arr.append(HanziConv.toTraditional(row[2]))

NameError: name 'HanziConv' is not defined

In [4]:
def simple2tradition(line):
    line = Converter('zh-hant').convert(line)
    return line

In [ ]:
fileTrainSeg=[]
SegTwoDimen = []
r = '\W+'
for i in range(len(all_)):
    all_[i] = simple2tradition(all_[i])
    words = re.sub(r,"",all_[i])
    wordarr = [word for word in jieba.cut(words)]
    SegTwoDimen.append(wordarr);
    fileTrainSeg.append([' '.join(wordarr)])

Building prefix dict from D:\program\semester_final\clean_data\dict.txt.big.txt ...
Loading model from cache C:\Users\PC-Henry\AppData\Local\Temp\jieba.u6179e01b4c6492e8310058a8757f6034.cache
Loading model cost 1.346 seconds.
Prefix dict has been built succesfully.


In [ ]:
SegTwoDimen[:5]

In [9]:
fileSegWordDonePath ='corpusSegDone.txt'
with open(fileSegWordDonePath,'wb') as fW:
    for i in range(len(fileTrainSeg)):
        fW.write(fileTrainSeg[i][0].encode('utf-8'))
        fW.write("\n".encode("utf-8"))
    print("完成!!")

完成!!


In [3]:
from gensim.models import word2vec
from gensim import corpora, models, similarities
from langconv import *

In [4]:
def PrintListChinese(list):
    for i in range(len(list)):
        print(list[i])
        
PrintListChinese(fileTrainSeg[10])

NameError: name 'fileTrainSeg' is not defined

In [12]:
sentences = word2vec.LineSentence("corpusSegDone.txt")
model = word2vec.Word2Vec(sentences, window=5, min_count=5, size=200, sg=0, workers=32, hs=1, iter=5)
model.save("word2vec.model")

In [13]:
model = word2vec.Word2Vec.load("word2vec.model")
corpora_dict_path = 'corpora_dict.dict'
dictionary = corpora.Dictionary(SegTwoDimen)
dictionary.save(corpora_dict_path)

In [14]:
corpora_dict_path = 'corpora_dict.dict'
tfidf_model_path = "./tfidf.model"
dictionary = corpora.Dictionary(SegTwoDimen)
dictionary.save(corpora_dict_path)

corpus = [dictionary.doc2bow(text) for text in SegTwoDimen]
corpora.MmCorpus.serialize('./corpus.mm', corpus) 

tfidf = models.TfidfModel(corpus)
tfidf.save(tfidf_model_path)

In [5]:
corpora_dict_path = 'corpora_dict.dict'
tfidf_model_path = "./tfidf.model"
corpus_path = './corpus.mm'
corpus = corpora.MmCorpus(corpus_path)
tfidf = models.TfidfModel.load(tfidf_model_path)
corpus_tfidf = tfidf[corpus]
dictionary = corpora.Dictionary.load(corpora_dict_path)
index = similarities.Similarity("./tempName",corpus_tfidf,len(dictionary.token2id))

In [ ]:
corpora_dict_path = 'corpora_dict.dict'
tfidf_model_path = "./tfidf.model"
corpus_path = './corpus.mm'
corpus = corpora.MmCorpus(corpus_path)
tfidf = models.TfidfModel.load(tfidf_model_path)
corpus_tfidf = tfidf[corpus]
dictionary = corpora.Dictionary.load(corpora_dict_path)
lsi = models.LsiModel(corpus, id2word=dictionary)
index = similarities.Similarity("./tempName",lsi[corpus],len(dictionary.token2id))

In [9]:
import jieba
import re
def clean_sentence(sentence):
    r = '[’!"#$%&\'()*+,./:;<=>?@，。?★、…【】\n《》\s？“”‘’�！[\\]^_`{|}~「」:]+'
    sentence = simple2tradition(sentence)
    words = re.sub(r,"",sentence)
    wordarr = [word for word in jieba.cut(words)]
    return " ".join(wordarr)

In [10]:
import numpy as np
test_doc = ["太陽能發電比燃煤發電便宜大陸宣佈做到了聯合新聞網","人家在做、我們在譙！太陽能發電比燃煤發電便宜 大陸宣布做到了！"]
test_doc = [clean_sentence(text) for text in test_doc]
new_vecs = [dictionary.doc2bow(test.split()) for test in test_doc]
new_vec_tfidf_ls = [tfidf[new_vec] for new_vec in new_vecs] 
sims = [index[new_vec_tfidf] for new_vec_tfidf in new_vec_tfidf_ls]
# sims = [sorted(list(zip(_id_ls,sim)),key= lambda tup:tup[1],reverse=True) for sim in sims]
# [print(sentence_dict[sim[0][0]]) for sim in sims]

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\PC-Henry\AppData\Local\Temp\jieba.cache
Loading model cost 0.711 seconds.
Prefix dict has been built succesfully.


In [12]:
index = np.argmax(np.array(sims[0]))
print(index)

0


In [20]:
print(sims)

[array([1.        , 0.8253497 , 0.        , ..., 0.        , 0.00233107,
       0.        ], dtype=float32), array([0.8253497 , 1.        , 0.        , ..., 0.        , 0.00403139,
       0.        ], dtype=float32)]


In [49]:
def gensimCalculation(d1, d2):
    text1 = [text for text in jieba.cut(d1)]
    texts = [[text for text in jieba.cut(d2)]]
    print(text1,texts)
    dictionary = corpora.Dictionary(texts)
    feature_cnt = len(dictionary.token2id)
    corpus = [dictionary.doc2bow(text) for text in texts]
    tfidf = models.TfidfModel(corpus)
    new_vec = dictionary.doc2bow(text1)
    index = similarities.SparseMatrixSimilarity(tfidf[corpus], num_features = feature_cnt)
    print(tfidf[new_vec])
    sim = index[tfidf[new_vec]]
    print(sim)

In [52]:
print(sims)

[array([0.78473645, 0.7787981 , 0.        , ..., 0.        , 0.        ,
       0.        ], dtype=float32), array([0.63801837, 0.8332558 , 0.        , ..., 0.        , 0.00105189,
       0.        ], dtype=float32)]


In [50]:
print(gensimCalculation("技術逐年進展，也讓太陽光電和陸域風電裝設成本下降","人家在做、我們在譙！太陽能發電比燃煤發電便宜 大陸宣布做到了！"))

['技術', '逐年', '進展', '，', '也', '讓', '太陽', '光電', '和', '陸域', '風電裝設', '成本', '下降'] [['人家', '在', '做', '、', '我們', '在', '譙', '！', '太陽能', '發電比', '燃煤', '發電', '便宜', ' ', '大陸', '宣布', '做到', '了', '！']]
[]
[0.]
None


In [10]:
print(sims)

[array([0.78473645, 0.7787981 , 0.        , ..., 0.        , 0.        ,
       0.        ], dtype=float32)]


In [2]:
from langconv import *
import csv

In [22]:
arr = []
with open("fake.csv",encoding="utf8") as file:
    rows = csv.reader(file)
    for row in rows:
        if(len(row) == 0):
            continue
        if()
        arr.append(row)

In [23]:
for ar in arr:
    ar[1] = Converter('zh-hant').convert(ar[1])
    ar[2] = Converter('zh-hant').convert(ar[2])

In [20]:
with open("fake.csv","w",encoding="utf8") as file:
    writer = csv.writer(file)
    for ar in arr:
        writer.writerow(ar)

In [12]:
from langconv import *
import csv
all_ = []
with open("compete_train.csv",encoding="utf8") as file:
    rows = csv.reader(file)
    next(rows,None)
    for row in rows:
        if(row[7]!="unrelated"):
            temp = [Converter('zh-hant').convert(sent).replace('"',"").replace('\xa0',"") for sent in row[3:5]]
            if(row[7] == "agreed"):
                temp.append(True)
            else:
                temp.append(False)
            all_.append(temp)

IndexError: list index out of range

In [11]:
all_[10:30]

[]

In [9]:
with open("compete_train.csv","w",encoding="utf8") as file:
    writer = csv.writer(file)
    for a in all_:
        writer.writerow(a)

In [9]:
import csv
all_ = []
with open("unfake_clean.csv",encoding="utf8") as file:
    rows = csv.reader(file)
    for row in rows:
        if(len(row) == 0 or row[0] == row[1] or abs(len(row[0])-len(row[1]))>15):
            continue
        all_.append(row)

In [10]:
with open("unfake_clean.csv","w",encoding="utf8") as file:
    writer = csv.writer(file)
    writer.writerows(all_)

In [9]:
import re
import csv

In [10]:
all_ = []
want = "家樂福60歲囉"
dont = ""
with open("fake_clean2.csv",encoding="utf8") as file:
    rows = csv.reader(file)
    for row in rows:
        if(len(row) == 0):
            continue
        if(re.search(want,row[1])):
            continue
        all_.append(row)
print(len(all_))

1353


In [5]:
with open("fake_clean2.csv","w",encoding="utf8") as file:
    writer = csv.writer(file)
    writer.writerows(all_)

In [14]:
all_ = []
with open("unfake_clean.csv",encoding="utf8") as file:
    rows = csv.reader(file)
    for row in rows:
        if(len(row) == 0):
            continue
        all_.append(row)
print(len(all_))

2777


In [1]:
from langconv import *
import pandas as pd

In [2]:
datas = pd.read_csv("train.csv")
datas.head()

,id,tid1,tid2,title1_zh,title2_zh,title1_en,title2_en,label
0,0,0,1,2017养老保险又新增两项，农村老人人人可申领，你领到了吗,警方辟谣“鸟巢大会每人领5万” 仍有老人坚持进京,There are two new old-age insurance benefits f...,"Police disprove ""bird's nest congress each per...",unrelated
1,3,2,3,"""你不来深圳，早晚你儿子也要来""，不出10年深圳人均GDP将超香港",深圳GDP首超香港？深圳统计局辟谣：只是差距在缩小,"""If you do not come to Shenzhen, sooner or lat...",Shenzhen's GDP outstrips Hong Kong? Shenzhen S...,unrelated
2,1,2,4,"""你不来深圳，早晚你儿子也要来""，不出10年深圳人均GDP将超香港",GDP首超香港？深圳澄清：还差一点点……,"""If you do not come to Shenzhen, sooner or lat...",The GDP overtopped Hong Kong? Shenzhen clarifi...,unrelated
3,2,2,5,"""你不来深圳，早晚你儿子也要来""，不出10年深圳人均GDP将超香港",去年深圳GDP首超香港？深圳统计局辟谣：还差611亿,"""If you do not come to Shenzhen, sooner or lat...",Shenzhen's GDP topped Hong Kong last year? She...,unrelated
4,9,6,7,"""用大蒜鉴别地沟油的方法,怎么鉴别地沟油",吃了30年食用油才知道，一片大蒜轻松鉴别地沟油,"""How to discriminate oil from gutter oil by me...",It took 30 years of cooking oil to know that o...,agreed


In [3]:
datas = datas.loc[:,["title1_zh","title2_zh","label"]]
datas.head()

,title1_zh,title2_zh,label
0,2017养老保险又新增两项，农村老人人人可申领，你领到了吗,警方辟谣“鸟巢大会每人领5万” 仍有老人坚持进京,unrelated
1,"""你不来深圳，早晚你儿子也要来""，不出10年深圳人均GDP将超香港",深圳GDP首超香港？深圳统计局辟谣：只是差距在缩小,unrelated
2,"""你不来深圳，早晚你儿子也要来""，不出10年深圳人均GDP将超香港",GDP首超香港？深圳澄清：还差一点点……,unrelated
3,"""你不来深圳，早晚你儿子也要来""，不出10年深圳人均GDP将超香港",去年深圳GDP首超香港？深圳统计局辟谣：还差611亿,unrelated
4,"""用大蒜鉴别地沟油的方法,怎么鉴别地沟油",吃了30年食用油才知道，一片大蒜轻松鉴别地沟油,agreed


In [4]:
def simtotra(text):
    return Converter('zh-hant').convert(text)
def stringfy(text):
    return str(text)

In [5]:
datas["title1_zh"] = datas.loc[:,"title1_zh"].apply(stringfy).apply(simtotra)
datas["title2_zh"] = datas.loc[:,"title2_zh"].apply(stringfy).apply(simtotra)
datas.head()

,title1_zh,title2_zh,label
0,2017養老保險又新增兩項，農村老人人人可申領，你領到了嗎,警方闢謠“鳥巢大會每人領5萬” 仍有老人堅持進京,unrelated
1,"""你不來深圳，早晚你兒子也要來""，不出10年深圳人均GDP將超香港",深圳GDP首超香港？深圳統計局闢謠：只是差距在縮小,unrelated
2,"""你不來深圳，早晚你兒子也要來""，不出10年深圳人均GDP將超香港",GDP首超香港？深圳澄清：還差一點點……,unrelated
3,"""你不來深圳，早晚你兒子也要來""，不出10年深圳人均GDP將超香港",去年深圳GDP首超香港？深圳統計局闢謠：還差611億,unrelated
4,"""用大蒜鑒別地溝油的方法,怎麼鑒別地溝油",吃了30年食用油才知道，一片大蒜輕鬆鑒別地溝油,agreed


In [7]:
datas.to_csv("train.csv")

*****

In [6]:
import pandas as pd
from langconv import *

In [7]:
def simtotra(text):
    return Converter('zh-hant').convert(text)
def stringfy(text):
    return str(text)

In [8]:
test = pd.read_csv("test.csv")
test = test.loc[:,["title1_zh","title2_zh"]]
test.head()

,title1_zh,title2_zh
0,萨拉赫人气爆棚!埃及总统大选未参选获百万选票 现任总统压力山大,辟谣！里昂官方否认费基尔加盟利物浦，难道是价格没谈拢？
1,萨达姆被捕后告诫美国的一句话，发人深思,10大最让美国人相信的荒诞谣言，如蜥蜴人掌控着美国
2,萨达姆此项计划没有此国破坏的话，美国还会对伊拉克发动战争吗,萨达姆被捕后告诫美国的一句话，发人深思
3,萨达姆被捕后告诫美国的一句话，发人深思,被绞刑处死的萨达姆是替身？他的此男人举动击破替身谣言！
4,萨达姆被捕后告诫美国的一句话，发人深思,中国川贝枇杷膏在美国受到热捧？纯属谣言！


In [11]:
test["title1_zh"] = test["title1_zh"].apply(stringfy).apply(simtotra)
test["title2_zh"] = test["title2_zh"].apply(stringfy).apply(simtotra)
test.head()

,title1_zh,title2_zh
0,薩拉赫人氣爆棚!埃及總統大選未參選獲百萬選票 現任總統壓力山大,闢謠！里昂官方否認費基爾加盟利物浦，難道是價格沒談攏？
1,薩達姆被捕後告誡美國的一句話，發人深思,10大最讓美國人相信的荒誕謠言，如蜥蜴人掌控著美國
2,薩達姆此項計劃沒有此國破壞的話，美國還會對伊拉克發動戰爭嗎,薩達姆被捕後告誡美國的一句話，發人深思
3,薩達姆被捕後告誡美國的一句話，發人深思,被絞刑處死的薩達姆是替身？他的此男人舉動擊破替身謠言！
4,薩達姆被捕後告誡美國的一句話，發人深思,中國川貝枇杷膏在美國受到熱捧？純屬謠言！


In [15]:
test = test[test["title2_zh"] != "nan"]

In [16]:
test.to_csv("../test_p.csv")

In [1]:
import pandas as pd